In [19]:
import boto3
import botocore
import pandas as pd
#declare bucket as a constant to use with AWS
BUCKET = "mindex-data-analytics-code-challenge"

#list of files to grab
files = ['bengals.csv', 'boyd_receiving.csv', 'chase_receiving.csv', 'higgins_receiving.csv']

#Ideally you'd want to use a DAO to handle connections
#For the sake of simplicity, I'll handle this within some util functions

#define the s3 resource object
s3 = boto3.resource('s3')

#Download util so I can take a look at the csv's locally
def download_csv(files):
    for file in files:
        #Keep the files name the same as the key
        try:
            s3.Bucket(BUCKET).download_file(file,file)
        except botocore.exceptions.ClientError as e:
            print('error downloading ' + file)
            print(e)

#Util for placing csv directly into a dataframe from s3
def get_dataframe(file):
    #There are some optional libraries that would let you read in a DF like:
    #pd.read_csv('s3://'+BUCKET+'/'+filename)
    #Since I don't see the dependency listed, I grab the CSV using the boto3 resource
    try:
        csv = s3.Object(BUCKET,file)
    except botocore.exceptions.ClientError as e:
        #An example of a better way to handle logging an exception could be sending A SNS notification
            print('error grabbing ' + file)
            print(e)

    return pd.read_csv(csv.get()['Body'])

In [9]:
#Just some code to download the csv's so I can check them out on my machine
download_csv(files)

In [20]:
#load each csv into a dataframe
bengals_record = get_dataframe(files[0])
boyd_receiving = get_dataframe(files[1])
chase_receiving = get_dataframe(files[2])
higgins_receiving = get_dataframe(files[3])